Download the dataset

In [1]:
%%script false
!curl -o datasets/big.txt https://norvig.com/big.txt

Couldn't find program: 'false'


Load the dataset

In [2]:
input_file_path = "datasets/big.txt"

with open(input_file_path, 'r', encoding='utf-8') as f:
    train_text = f.read()

print(f"Loaded dataset with {len(train_text)} characters.")

Loaded dataset with 6488666 characters.


In [3]:
print(train_text[:1000])

The Project Gutenberg EBook of The Adventures of Sherlock Holmes
by Sir Arthur Conan Doyle
(#15 in our series by Sir Arthur Conan Doyle)

Copyright laws are changing all over the world. Be sure to check the
copyright laws for your country before downloading or redistributing
this or any other Project Gutenberg eBook.

This header should be the first thing seen when viewing this Project
Gutenberg file.  Please do not remove it.  Do not change or edit the
header without written permission.

Please read the "legal small print," and other information about the
eBook and Project Gutenberg at the bottom of this file.  Included is
important information about your specific rights and restrictions in
how the file may be used.  You can also find out about how to make a
donation to Project Gutenberg, and how to get involved.


**Welcome To The World of Free Plain Vanilla Electronic Texts**

**eBooks Readable By Both Humans and By Computers, Since 1971**

*****These eBooks Were Prepared By Thousan

Isolate unique chars

In [4]:
chars = sorted(list(set(train_text)))
num_unique_chars = len(chars)
print(''.join(chars))
print(num_unique_chars)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz|~
93


Make a simple char tokenizeer

In [5]:
char_to_int = { char:i for i,char in enumerate(chars) }
int_to_char = { i:char for i,char in enumerate(chars) }
encode = lambda str: [char_to_int[char] for char in str]
decode = lambda ints: ''.join([int_to_char[i] for i in ints])

print(decode(encode("hello world!")))

hello world!


In [6]:
%%script false
import tiktoken
enc = tiktoken.get_encoding('gpt2')

encode = enc.encode
decode = enc.decode

Couldn't find program: 'false'


In [7]:
import torch
data = torch.tensor(encode(train_text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([6488666]) torch.int64
tensor([54, 72, 69,  2, 50, 82, 79, 74, 69, 67, 84,  2, 41, 85, 84, 69, 78, 66,
        69, 82, 71,  2, 39, 36, 79, 79, 75,  2, 79, 70,  2, 54, 72, 69,  2, 35,
        68, 86, 69, 78, 84, 85, 82, 69, 83,  2, 79, 70,  2, 53, 72, 69, 82, 76,
        79, 67, 75,  2, 42, 79, 76, 77, 69, 83,  1, 66, 89,  2, 53, 73, 82,  2,
        35, 82, 84, 72, 85, 82,  2, 37, 79, 78, 65, 78,  2, 38, 79, 89, 76, 69,
         1, 10,  5, 19, 23,  2, 73, 78,  2, 79])


Split data into training and validation sets based on randomly selected chunks

In [20]:
import torch, random

newline_token = encode("\n")
space_token = encode(" ")

val_fraction = 0.1
num_chunks   = 5

total_len = data.size(0)
val_len   = int(val_fraction * total_len)
chunk_len = val_len // num_chunks
remainder = val_len % num_chunks

# split the entire dataset into chunks
chunks = []
idx = 0
while idx + chunk_len <= total_len:
    chunks.append(data[idx : idx + chunk_len])
    idx += chunk_len

# the leftover chunk (if any) after slicing out as many `chunk_len` blocks as possible. We always put this leftover into training by default
leftover = data[idx:]

val_indices = sorted(random.sample(range(len(chunks)), num_chunks))

# build the validation data (insert newline_token only between non-consecutive chunks)
val_data = []
prev_idx = -2  # something not adjacent to first pick 
for i in val_indices:
    # if the current chunk is NOT directly after the previous one and val_data already has content,
    # then insert a newline token
    if i != prev_idx + 1 and len(val_data) > 0:
        val_data.append(torch.tensor(newline_token, dtype=torch.int64))
    val_data.append(chunks[i])
    prev_idx = i

print(val_data)

# concatenate chosen val chunks
val_data = torch.cat(val_data) if val_data else torch.empty(0, dtype=torch.int64)

# if we have a remainder that is significant, tack data from train onto the end. This is ugly and bad and should be changed
# this ensures total validation tokens == val_len but can also result in a small amount of overlap (up to chunk_length -1 tokens) between val and train
if remainder > 100:
    val_data = torch.cat((val_data, data[-remainder:]))

# everything else is training: the unchosen chunks plus leftover
train_indices = set(range(len(chunks))) - set(val_indices)
train_data = [chunks[i] for i in train_indices]
train_data.append(leftover)  # leftover always goes to training
train_data = torch.cat(train_data) if train_data else torch.empty(0, dtype=torch.int64)

print("Training data size  :", train_data.size(0))
print("Validation data size:", val_data.size(0))
print("Total data size     :", total_len)

[tensor([54, 72, 69,  ..., 82, 16,  2]), tensor([53, 72, 69,  ..., 83, 85, 82]), tensor([1]), tensor([67, 65, 76,  ..., 84,  2, 79]), tensor([1]), tensor([73, 66, 76,  ..., 80, 65, 82]), tensor([1]), tensor([77, 69, 84,  ..., 78, 68,  2])]
Training data size  : 5839801
Validation data size: 648868
Total data size     : 6488666


In [21]:
block_size = 15
train_data[:block_size+1]

tensor([80, 82, 73, 83, 69, 16,  2, 35, 83, 84, 79, 78, 73, 83, 72, 77])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1].tolist()
    target = [y[t].item()]
    print(f"when input is {decode(context)} the target: {decode(target)}")

TypeError: 'int' object is not iterable